In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()
spark

In [3]:
training= spark.read.csv('test1.csv',header=True,inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sundhansh| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 21|         2| 18000|
+---------+---+----------+------+



In [4]:
 training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

## [Age,Experience]-----> new feature -----> independent feature

In [11]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")

In [12]:
output=featureassembler.transform(training)

In [13]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|Age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sundhansh| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 21|         2| 18000|          [21.0,2.0]|
+---------+---+----------+------+--------------------+



In [14]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [17]:
finalized_data=output.select("Independent Features","Salary")

In [18]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [21.0,2.0]| 18000|
+--------------------+------+



In [21]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features',labelCol='Salary')
regressor=regressor.fit(train_data)

In [22]:
regressor.coefficients

DenseVector([-130.4348, 1753.6232])

In [23]:
regressor.intercept

16623.188405797016

In [24]:
pred_results=regressor.evaluate(test_data)

In [25]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000|18753.623188405792|
|          [30.0,8.0]| 25000|26739.130434782608|
+--------------------+------+------------------+



In [27]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1492.7536231884078, 2289014.912833443)